# Initialize Model

In [1]:
from package import embedding, llm, agent_llm

# Import Lib

In [18]:
import json

In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate

In [3]:
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser

In [4]:
from operator import itemgetter
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda

In [5]:
from langchain.docstore.document import Document

In [6]:
from langchain_community.vectorstores import DocArrayInMemorySearch

In [4]:
from package.utils import format_docs, prompt_2_str

# UDF

In [7]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

In [8]:
def prompt_2_str(prompt):
    return prompt.text

# Test

In [11]:
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
# from langchain_openai import OpenAI

# model = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.0)


# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Badly formed question!")
        return field


# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# And a query intended to prompt a language model to populate the data structure.
prompt_and_model = prompt | llm
output = prompt_and_model.invoke({"query": "Tell me a joke."})
response = parser.invoke(output)

In [12]:
response

Joke(setup="Why don't scientists trust atoms?", punchline='Because they make up everything!')

In [13]:
print(response)

setup="Why don't scientists trust atoms?" punchline='Because they make up everything!'


In [22]:
json.loads(response.json())

{'setup': "Why don't scientists trust atoms?",
 'punchline': 'Because they make up everything!'}

In [23]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"setup": {"title": "Setup", "description": "question to set up a joke", "type": "string"}, "punchline": {"title": "Punchline", "description": "answer to resolve the joke", "type": "string"}}, "required": ["setup", "punchline"]}
```
